In [2]:
%pylab inline
from numpy.lib.recfunctions import append_fields


Populating the interactive namespace from numpy and matplotlib


In [2]:
avs = np.array([0,1,2,3,5,10,20])
x = []
for av in avs:
    temp = np.load("../data/GDR3/gaiadr3_%dext.npy" %(av))
    temp = append_fields(temp,"parsec_index",np.zeros(shape = len(temp),dtype = np.int32),usemask=False)
    x.append(temp)
print(len(x[0]),x[0].dtype)

8102858 [('meh_ini', '<f8'), ('log_age', '<f8'), ('m_ini', '<f8'), ('m_act', '<f8'), ('log_lum', '<f8'), ('log_teff', '<f8'), ('log_grav', '<f8'), ('gaia_g', '<f8'), ('gaia_bpbr', '<f8'), ('gaia_bpft', '<f8'), ('gaia_rp', '<f8'), ('gaia_rvs', '<f8'), ('parsec_index', '<i4')]


In [3]:
for name in x[0].dtype.names:
    print(name,x[0][name],len(np.unique(x[0][name])))

meh_ini [-1.49213 -1.49213 -1.49213 ...  0.33631  0.33631  0.33631] 75
log_age [ 6.6      6.6      6.6     ... 10.12004 10.12004 10.12004] 177
m_ini [0.09       0.0956148  0.1        ... 5.80000019 5.99999952 6.19999933] 6276478
m_act [0.09  0.096 0.1   ... 0.94  0.961 0.962] 53226
log_lum [-1.531 -1.502 -1.481 ... -4.601 -4.603 -4.604] 10836
log_teff [3.5276 3.5305 3.5326 ... 3.6298 3.6321 3.6319] 27662
log_grav [3.986 3.995 4.001 ... 8.484 8.505 8.505] 12072
gaia_g [ 9.192  9.106  9.043 ... 16.513 16.507 16.51 ] 52080
gaia_bpbr [10.284 10.181 10.108 ... 17.284 17.27  17.273] 67368
gaia_bpft [10.319 10.217 10.143 ... 17.324 17.31  17.313] 67036
gaia_rp [ 8.163  8.085  8.027 ... 15.66  15.66  15.662] 49227
gaia_rvs [ 7.692  7.62   7.566 ... 15.318 15.323 15.325] 43937
parsec_index [0 0 0 ... 0 0 0] 1


In [4]:
def return_index_feh(feh):
        dfeh = 0.05
        offset = 1.5
        return(int((feh+offset)/dfeh))
min_value = min(x[0]['meh_ini'])
max_value = max(x[0]['meh_ini'])
min_index_feh = return_index_feh(min_value)
max_index_feh = return_index_feh(max_value)
stretch = max_index_feh - min_index_feh
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_feh, max_index_feh)
print('dimension cut into %d pieces' %(stretch))

def return_index_teff(teff):
        dteff = 0.01
        return(int(teff/dteff))
min_value = min(x[0]['log_teff'])
max_value = max(x[0]['log_teff'])
min_index_teff = return_index_teff(min_value)
max_index_teff = return_index_teff(max_value)
stretch = max_index_teff - min_index_teff
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_teff, max_index_teff)
print('dimension cut into %d pieces' %(stretch))

def return_index_lum(lum):
        dlum = 0.02
        offset = 5.
        return(int((lum+offset)/dlum))

min_value = min(x[0]['log_lum'])
max_value = max(x[0]['log_lum'])
min_index_lum = return_index_lum(min_value)
max_index_lum = return_index_lum(max_value)
stretch = max_index_lum - min_index_lum
print('lum values in parsec')
print('lum min max value: ', min_value, max_value )
print('lum min max index: ', min_index_lum, max_index_lum )
print('dimension cut into %d pieces' %(stretch))

def return_index(feh,teff,lum):
        """
        feh given in dex
        teff given in log teff
        lum given in log lum
        """
        index_feh = return_index_feh(feh)
        if index_feh < min_index_feh:
            index_feh = min_index_feh
        elif index_feh > max_index_feh:
            index_feh = max_index_feh

        index_teff = return_index_teff(teff)
        if index_teff < min_index_teff:
            index_teff = min_index_teff
        elif index_teff > max_index_teff:
            index_teff = max_index_teff
        index_teff *= 1000

        index_lum = return_index_lum(lum)
        if index_lum > max_index_lum:
            index_lum = max_index_lum
        elif index_lum < min_index_lum:
            index_lum = min_index_lum
        index_lum *= 1000 * 1000
        assert(index_feh >= 0)
        assert(index_teff >= 0)
        assert(index_lum >= 0)
        return (index_feh + index_teff + index_lum)

teff values in parsec
teff min max value:  -1.49213 0.33631
teff min max index:  0 36
dimension cut into 36 pieces
teff values in parsec
teff min max value:  1.666 5.6843
teff min max index:  166 568
dimension cut into 402 pieces
lum values in parsec
lum min max value:  -4.604 6.24
lum min max index:  19 562
dimension cut into 543 pieces


In [5]:
print('indexing isochrones')
for i in range(len(x[0])):
    x[0]["parsec_index"][i] = return_index(x[0]["meh_ini"][i],x[0]["log_teff"][i],x[0]["log_lum"][i])

indexing isochrones


In [6]:
# All need to be sorted by parsec_index
sort_ind = np.argsort(x[0]['parsec_index'], kind = 'mergesort')
for i in range(len(avs)):
    x[i] = x[i][sort_ind]

In [7]:
g_ext = np.zeros(shape=(len(x[0]),len(x)))
bp_ext = np.zeros(shape=(len(x[0]),len(x)))
bpbr_ext = np.zeros(shape=(len(x[0]),len(x)))
rp_ext = np.zeros(shape=(len(x[0]),len(x)))
rvs_ext = np.zeros(shape=(len(x[0]),len(x)))
for i,item in enumerate(x):
    print(i,item[0])
    g_ext[:,i] = item["gaia_g"]
    bp_ext[:,i] = item["gaia_bpft"]
    bpbr_ext[:,i] = item["gaia_bpbr"]
    rp_ext[:,i] = item["gaia_rp"]
    rvs_ext[:,i] = item["gaia_rvs"]

0 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 16.509, 17.278, 17.318, 15.657, 15.317, 19362036)
1 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 17.266, 18.268, 18.306, 16.279, 15.84, 0)
2 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 17.987, 19.236, 19.27, 16.888, 16.362, 0)
3 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 18.65, 20.17, 20.199, 17.47, 16.884, 0)
4 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 19.978, 22.038, 22.058, 18.635, 17.929, 0)
5 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 22.909, 26.504, 26.492, 21.328, 20.538, 0)
6 (0.3, 10.12004, 5.73621607, 0.937, -4.601, 3.6296, 8.481, 28.026, 35.091, 35.025, 26.197, 25.751, 0)


In [8]:
# show the extinction law and on which parameters it is dependent
g_ext = g_ext - g_ext[:,0,None]
rvs_ext = rvs_ext - rvs_ext[:,0,None]
bp_ext = bp_ext - bp_ext[:,0,None]
rp_ext = rp_ext - rp_ext[:,0,None]
bpbr_ext = bpbr_ext - bpbr_ext[:,0,None]

In [9]:
ext = np.zeros(shape = (len(np.unique(x[0]["parsec_index"])),5,6),dtype = np.float32)
ext_std = np.zeros_like(ext)
parsec = np.zeros(shape =  (len(np.unique(x[0]["parsec_index"]))),dtype = x[0].dtype)
parsec_std = np.zeros(shape =  (len(np.unique(x[0]["parsec_index"]))),dtype = x[0].dtype)

In [10]:
parsec["parsec_index"] = np.unique(x[0]["parsec_index"])
parsec_std["parsec_index"] = np.unique(x[0]["parsec_index"])

In [11]:
indexing = np.hstack((0,np.searchsorted(x[0]['parsec_index'],parsec['parsec_index'], side='right')))
print(indexing)
for i in range(len(indexing)-1):
    if i%10000==0:
        print(i, len(indexing))
    lower = indexing[i]
    upper = indexing[i+1]
    #parsec['parsec_index'][i]
    #x[0]['parsec_index'][lower:upper]
    temp = x[0][lower:upper]
    g_temp = g_ext[lower:upper]
    bp_temp = bp_ext[lower:upper]
    bpbr_temp = bpbr_ext[lower:upper]
    rp_temp = rp_ext[lower:upper]
    rvs_temp = rvs_ext[lower:upper]
    for jtem in parsec.dtype.names:
        if jtem == "parsec_index":
            assert(parsec["parsec_index"][i] == temp["parsec_index"][0])
        else:
            parsec[jtem][i] = np.median(temp[jtem])
            parsec_std[jtem][i] = np.std(temp[jtem])
    for t in np.arange(6):
        ext[i,0,t] = np.median(g_temp[:,t+1])
        ext[i,1,t] = np.median(bp_temp[:,t+1])
        ext[i,2,t] = np.median(bpbr_temp[:,t+1])
        ext[i,3,t] = np.median(rp_temp[:,t+1])
        ext[i,4,t] = np.median(rvs_temp[:,t+1])
        
        ext_std[i,0,t] = np.std(g_temp[:,t+1])
        ext_std[i,1,t] = np.std(bp_temp[:,t+1])
        ext_std[i,2,t] = np.std(bpbr_temp[:,t+1])
        ext_std[i,3,t] = np.std(rp_temp[:,t+1])
        ext_std[i,4,t] = np.std(rvs_temp[:,t+1])


[      0       3       4 ... 8102856 8102857 8102858]
0 893260
100 893260
200 893260
300 893260
400 893260
500 893260
600 893260
700 893260
800 893260
900 893260
1000 893260
1100 893260
1200 893260
1300 893260
1400 893260
1500 893260
1600 893260
1700 893260
1800 893260
1900 893260
2000 893260
2100 893260
2200 893260
2300 893260
2400 893260
2500 893260
2600 893260
2700 893260
2800 893260
2900 893260
3000 893260
3100 893260
3200 893260
3300 893260
3400 893260
3500 893260
3600 893260
3700 893260
3800 893260
3900 893260
4000 893260
4100 893260
4200 893260
4300 893260
4400 893260
4500 893260
4600 893260
4700 893260
4800 893260
4900 893260
5000 893260
5100 893260
5200 893260
5300 893260
5400 893260
5500 893260
5600 893260
5700 893260
5800 893260
5900 893260
6000 893260
6100 893260
6200 893260
6300 893260
6400 893260
6500 893260
6600 893260
6700 893260
6800 893260
6900 893260
7000 893260
7100 893260
7200 893260
7300 893260
7400 893260
7500 893260
7600 893260
7700 893260
7800 893260
7900 89326

63500 893260
63600 893260
63700 893260
63800 893260
63900 893260
64000 893260
64100 893260
64200 893260
64300 893260
64400 893260
64500 893260
64600 893260
64700 893260
64800 893260
64900 893260
65000 893260
65100 893260
65200 893260
65300 893260
65400 893260
65500 893260
65600 893260
65700 893260
65800 893260
65900 893260
66000 893260
66100 893260
66200 893260
66300 893260
66400 893260
66500 893260
66600 893260
66700 893260
66800 893260
66900 893260
67000 893260
67100 893260
67200 893260
67300 893260
67400 893260
67500 893260
67600 893260
67700 893260
67800 893260
67900 893260
68000 893260
68100 893260
68200 893260
68300 893260
68400 893260
68500 893260
68600 893260
68700 893260
68800 893260
68900 893260
69000 893260
69100 893260
69200 893260
69300 893260
69400 893260
69500 893260
69600 893260
69700 893260
69800 893260
69900 893260
70000 893260
70100 893260
70200 893260
70300 893260
70400 893260
70500 893260
70600 893260
70700 893260
70800 893260
70900 893260
71000 893260
71100 893260

124700 893260
124800 893260
124900 893260
125000 893260
125100 893260
125200 893260
125300 893260
125400 893260
125500 893260
125600 893260
125700 893260
125800 893260
125900 893260
126000 893260
126100 893260
126200 893260
126300 893260
126400 893260
126500 893260
126600 893260
126700 893260
126800 893260
126900 893260
127000 893260
127100 893260
127200 893260
127300 893260
127400 893260
127500 893260
127600 893260
127700 893260
127800 893260
127900 893260
128000 893260
128100 893260
128200 893260
128300 893260
128400 893260
128500 893260
128600 893260
128700 893260
128800 893260
128900 893260
129000 893260
129100 893260
129200 893260
129300 893260
129400 893260
129500 893260
129600 893260
129700 893260
129800 893260
129900 893260
130000 893260
130100 893260
130200 893260
130300 893260
130400 893260
130500 893260
130600 893260
130700 893260
130800 893260
130900 893260
131000 893260
131100 893260
131200 893260
131300 893260
131400 893260
131500 893260
131600 893260
131700 893260
131800

183300 893260
183400 893260
183500 893260
183600 893260
183700 893260
183800 893260
183900 893260
184000 893260
184100 893260
184200 893260
184300 893260
184400 893260
184500 893260
184600 893260
184700 893260
184800 893260
184900 893260
185000 893260
185100 893260
185200 893260
185300 893260
185400 893260
185500 893260
185600 893260
185700 893260
185800 893260
185900 893260
186000 893260
186100 893260
186200 893260
186300 893260
186400 893260
186500 893260
186600 893260
186700 893260
186800 893260
186900 893260
187000 893260
187100 893260
187200 893260
187300 893260
187400 893260
187500 893260
187600 893260
187700 893260
187800 893260
187900 893260
188000 893260
188100 893260
188200 893260
188300 893260
188400 893260
188500 893260
188600 893260
188700 893260
188800 893260
188900 893260
189000 893260
189100 893260
189200 893260
189300 893260
189400 893260
189500 893260
189600 893260
189700 893260
189800 893260
189900 893260
190000 893260
190100 893260
190200 893260
190300 893260
190400

241900 893260
242000 893260
242100 893260
242200 893260
242300 893260
242400 893260
242500 893260
242600 893260
242700 893260
242800 893260
242900 893260
243000 893260
243100 893260
243200 893260
243300 893260
243400 893260
243500 893260
243600 893260
243700 893260
243800 893260
243900 893260
244000 893260
244100 893260
244200 893260
244300 893260
244400 893260
244500 893260
244600 893260
244700 893260
244800 893260
244900 893260
245000 893260
245100 893260
245200 893260
245300 893260
245400 893260
245500 893260
245600 893260
245700 893260
245800 893260
245900 893260
246000 893260
246100 893260
246200 893260
246300 893260
246400 893260
246500 893260
246600 893260
246700 893260
246800 893260
246900 893260
247000 893260
247100 893260
247200 893260
247300 893260
247400 893260
247500 893260
247600 893260
247700 893260
247800 893260
247900 893260
248000 893260
248100 893260
248200 893260
248300 893260
248400 893260
248500 893260
248600 893260
248700 893260
248800 893260
248900 893260
249000

...

652100 893260
652200 893260
652300 893260
652400 893260
652500 893260
652600 893260
652700 893260
652800 893260
652900 893260
653000 893260
653100 893260
653200 893260
653300 893260
653400 893260
653500 893260
653600 893260
653700 893260
653800 893260
653900 893260
654000 893260
654100 893260
654200 893260
654300 893260
654400 893260
654500 893260
654600 893260
654700 893260
654800 893260
654900 893260
655000 893260
655100 893260
655200 893260
655300 893260
655400 893260
655500 893260
655600 893260
655700 893260
655800 893260
655900 893260
656000 893260
656100 893260
656200 893260
656300 893260
656400 893260
656500 893260
656600 893260
656700 893260
656800 893260
656900 893260
657000 893260
657100 893260
657200 893260
657300 893260
657400 893260
657500 893260
657600 893260
657700 893260
657800 893260
657900 893260
658000 893260
658100 893260
658200 893260
658300 893260
658400 893260
658500 893260
658600 893260
658700 893260
658800 893260
658900 893260
659000 893260
659100 893260
659200

710700 893260
710800 893260
710900 893260
711000 893260
711100 893260
711200 893260
711300 893260
711400 893260
711500 893260
711600 893260
711700 893260
711800 893260
711900 893260
712000 893260
712100 893260
712200 893260
712300 893260
712400 893260
712500 893260
712600 893260
712700 893260
712800 893260
712900 893260
713000 893260
713100 893260
713200 893260
713300 893260
713400 893260
713500 893260
713600 893260
713700 893260
713800 893260
713900 893260
714000 893260
714100 893260
714200 893260
714300 893260
714400 893260
714500 893260
714600 893260
714700 893260
714800 893260
714900 893260
715000 893260
715100 893260
715200 893260
715300 893260
715400 893260
715500 893260
715600 893260
715700 893260
715800 893260
715900 893260
716000 893260
716100 893260
716200 893260
716300 893260
716400 893260
716500 893260
716600 893260
716700 893260
716800 893260
716900 893260
717000 893260
717100 893260
717200 893260
717300 893260
717400 893260
717500 893260
717600 893260
717700 893260
717800

769300 893260
769400 893260
769500 893260
769600 893260
769700 893260
769800 893260
769900 893260
770000 893260
770100 893260
770200 893260
770300 893260
770400 893260
770500 893260
770600 893260
770700 893260
770800 893260
770900 893260
771000 893260
771100 893260
771200 893260
771300 893260
771400 893260
771500 893260
771600 893260
771700 893260
771800 893260
771900 893260
772000 893260
772100 893260
772200 893260
772300 893260
772400 893260
772500 893260
772600 893260
772700 893260
772800 893260
772900 893260
773000 893260
773100 893260
773200 893260
773300 893260
773400 893260
773500 893260
773600 893260
773700 893260
773800 893260
773900 893260
774000 893260
774100 893260
774200 893260
774300 893260
774400 893260
774500 893260
774600 893260
774700 893260
774800 893260
774900 893260
775000 893260
775100 893260
775200 893260
775300 893260
775400 893260
775500 893260
775600 893260
775700 893260
775800 893260
775900 893260
776000 893260
776100 893260
776200 893260
776300 893260
776400

827900 893260
828000 893260
828100 893260
828200 893260
828300 893260
828400 893260
828500 893260
828600 893260
828700 893260
828800 893260
828900 893260
829000 893260
829100 893260
829200 893260
829300 893260
829400 893260
829500 893260
829600 893260
829700 893260
829800 893260
829900 893260
830000 893260
830100 893260
830200 893260
830300 893260
830400 893260
830500 893260
830600 893260
830700 893260
830800 893260
830900 893260
831000 893260
831100 893260
831200 893260
831300 893260
831400 893260
831500 893260
831600 893260
831700 893260
831800 893260
831900 893260
832000 893260
832100 893260
832200 893260
832300 893260
832400 893260
832500 893260
832600 893260
832700 893260
832800 893260
832900 893260
833000 893260
833100 893260
833200 893260
833300 893260
833400 893260
833500 893260
833600 893260
833700 893260
833800 893260
833900 893260
834000 893260
834100 893260
834200 893260
834300 893260
834400 893260
834500 893260
834600 893260
834700 893260
834800 893260
834900 893260
835000

886500 893260
886600 893260
886700 893260
886800 893260
886900 893260
887000 893260
887100 893260
887200 893260
887300 893260
887400 893260
887500 893260
887600 893260
887700 893260
887800 893260
887900 893260
888000 893260
888100 893260
888200 893260
888300 893260
888400 893260
888500 893260
888600 893260
888700 893260
888800 893260
888900 893260
889000 893260
889100 893260
889200 893260
889300 893260
889400 893260
889500 893260
889600 893260
889700 893260
889800 893260
889900 893260
890000 893260
890100 893260
890200 893260
890300 893260
890400 893260
890500 893260
890600 893260
890700 893260
890800 893260
890900 893260
891000 893260
891100 893260
891200 893260
891300 893260
891400 893260
891500 893260
891600 893260
891700 893260
891800 893260
891900 893260
892000 893260
892100 893260
892200 893260
892300 893260
892400 893260
892500 893260
892600 893260
892700 893260
892800 893260
892900 893260
893000 893260
893100 893260
893200 893260


In [12]:
np.save('ext.npy',ext)
np.save('ext_std.npy', ext_std)
np.save('parsec.npy', parsec)
np.save('parsec_std.npy', parsec_std)

In [13]:
# check that the stds are small and also that Galaxia falls onto those parsec indexes.